<a href="https://colab.research.google.com/github/asdjimespoir/Skin_Lesion_Segmentation_Cancer_Detection/blob/main/Yolo_Detection_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tâche 2 : Détection et segmentation de lésions cutanées avec YOLOv8**

>Dans cette tâche nous allons voir comment entrainer la detection d'objets et la segmentation d'image sur un jeux de données personnalisé avec Yolo. Mais avant tout, `c'est quoi YOLOv8 ?` `Qu'est-ce que la détection d'objets ?` `Qu'est que la segmentation d'image ?` sont là des questions auxquelles nous devons répondre.

#### **C'est quoi YOLOv8 ?**

Alors `YOLOv8`  est un modèle de détection d'objets de pointe utilisé dans les tâches de vision par ordinateur. YOLO (You Only Look Once) est un type de réseau de détection d'objets rapide et efficace, ce qui en fait un choix populaire pour la détection d'objets en temps réel dans les applications de vision par ordinateur. YOLOv8 est la huitième version du modèle YOLO et est connu pour être l'un des modèles de détection d'objets les plus performants en termes de précision et de vitesse.

Dans YOLO, une image est divisée en une grille de cellules et pour chaque cellule, le modèle prédit les boîtes englobantes et les probabilités de classe des objets qui pourraient être présents dans la cellule. L'architecture YOLO est conçue pour être rapide, ce qui la rend adaptée à la détection d'objets en temps réel dans diverses applications de vision par ordinateur telles que les véhicules autonomes, la vidéosurveillance et la récupération d'images.

#### **Qu'est-ce que la détection d'objets ?**

La détection d'objets est un domaine de la vision par ordinateur qui a pour but de localiser et d'identifier des objets spécifiques dans une image ou une vidéo. Il s'agit d'une tâche de reconnaissance d'images dans laquelle le système doit être capable de détecter la présence et l'emplacement des objets dans l'image, ainsi que de les classer en différentes catégories, telles que les voitures, les personnes, les animaux, etc.

La détection d'objets est largement utilisée dans de nombreuses applications, notamment la reconnaissance de la circulation, la surveillance vidéo, la reconnaissance de la scène, la reconnaissance de la forme, la reconnaissance d'images, etc. Les algorithmes de détection d'objets peuvent être basés sur des techniques telles que les réseaux de neurones convolutionnels, les méthodes basées sur les caractéristiques, les méthodes basées sur les modèles et les méthodes de détection de bord. Les réseaux de neurones convolutionnels, tels que YOLO et Faster R-CNN, sont actuellement considérés comme les méthodes les plus avancées pour la détection d'objets en raison de leur précision et de leur efficacité en termes de temps de traitement.

 #### **Qu'est que la segmentation d'image ?**

La segmentation d'image est un domaine de la vision par ordinateur qui consiste à diviser une image en plusieurs régions homogènes, appelées segments, en fonction de certains critères. Chaque segment représente une région d'intérêt distincte de l'image, comme un objet, une personne, un animal, etc.

La segmentation d'image est un prétraitement important pour de nombreuses tâches en vision par ordinateur, telles que la reconnaissance d'objets, la reconnaissance de la forme, la reconstruction 3D, etc. Les algorithmes de segmentation d'image peuvent être basés sur des techniques telles que les réseaux de neurones, les algorithmes de filtrage, les algorithmes basés sur les modèles, etc. Les réseaux de neurones, tels que U-Net et SegNet, sont actuellement considérés comme les méthodes les plus avancées pour la segmentation d'image en raison de leur capacité à capturer des caractéristiques complexes et à générer des résultats précis.

In [ ]:
%%capture
!pip install ultralytics==8.0.20

In [ ]:
import os
HOME = os.getcwd()

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
#!yolo mode=checks
from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
from google.colab import drive
drive.mount('/content/Drive')

## **Préparation du Dataset et importation des modules nécessaires**

Installons la bibliothèque Python `Roboflow` et utilisons l'API associée pour accéder à notre dataset labellisé sur Roboflow et téléchargeons-le au format yolov8.


In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="bOBnXq0Z83ShJXJhxdl4")
project = rf.workspace("asdjim-espoir-guelmian-vi-guelmian-vi").project("cancer-qshvo")
dataset = project.version(4).download("yolov8")


## **Détection d'object**

### Formation

Entraînons le modèle YOLOv8s sur un notre dataset importé pour la détection d'objets en utilisant l'outil de ligne de commande YOLO et les paramètres suivants :

- `task=detect` indique que la tâche consiste en une détection d'objets.

- `mode=train` indique que nous souhaitons entraîner un modèle.

- `model=yolov8s.pt` indique que nous souhaitons utiliser le modèle YOLOv8s.

- `data={dataset.location}/data.yaml` fournit l'emplacement des données d'entraînement à partir de la variable dataset définie précédemment.

- `epochs=25` indique que nous souhaitons entraîner le modèle sur 25 époques.

- `imgsz=800` indique la taille de l'image utilisée pour l'entraînement.

- `plots=True` indique que nous souhaitons afficher des graphiques de suivi de la performance du modèle au fil de l'entraînement.


In [ ]:
%cd {HOME}
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=50 imgsz=800 plots=True

In [ ]:
!ls {HOME}/runs/detect/train2/

Affichons la matrice de confusion

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train2/confusion_matrix.png', width=600)

Affichons les sorties plots des metriques pour les données de formation et de validation

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train2/results.png', width=600)

Resultat des prédiction sur les données de validation

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train2/val_batch0_pred.jpg', width=600)

### Validation du modèle

In [ ]:
%cd {HOME}
!yolo task=detect mode=val model={HOME}/runs/detect/train2/weights/best.pt data={dataset.location}/data.yaml

### Inférence avec le modèle personnalisé

Prédiction sur les données de test avec notre modèle final

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train2/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

**Jetons un coup d'oeil à quelques résultats.**

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[:]:
      display(Image(filename=image_path, width=300))
      print("\n")

### Déploiement du modèle

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=f"{HOME}/runs/detect/train2/")

In [ ]:
#Exécutez des inférences sur votre modèle à l'aide d'une API en nuage persistante et à mise à l'échelle automatique.

#Chargement du modèle
model = project.version(dataset.version).model

#Choisir une image aléatoire de l'ensemble de test
import os, random
test_set_loc = dataset.location + "/test/images/"
random_test_image = random.choice(os.listdir(test_set_loc))
print("running inference on " + random_test_image)

pred = model.predict(test_set_loc + random_test_image, confidence=40, overlap=30).json()
pred

### 🐍 SDK Python

Le moyen le plus simple d'utiliser YOLOv8 directement dans un environnement Python.

In [ ]:
model = YOLO(f'{HOME}/yolov8s.pt')#/runs/detect/train2/weights/best.pt
results = model.predict(source='/content/Drive/MyDrive/DL/HAM/Part_1/ISIC_0027820.jpg', conf=0.25)

In [ ]:
results[0].boxes.xyxy

In [ ]:
results[0].boxes.conf

In [ ]:
results[0].boxes.cls

## **Segmentation instance**

In [ ]:
!yolo task=segment mode=predict model=yolov8s-seg.pt conf=0.4 source='/content/Drive/MyDrive/DL/HAM/Part_1/ISIC_0024306.jpg' save=True

In [ ]:
%cd {HOME}
Image(filename='runs/segment/predict6/ISIC_0024306.jpg', height=600)

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="bOBnXq0Z83ShJXJhxdl4")
project = rf.workspace("nicolai-hoirup-nielsen").project("cupsegmentation")
dataset = project.version(1).download("yolov8")

In [ ]:
#%cd {HOME}
!yolo task=segment mode=train model=yolov8l-seg.pt data={dataset.location}/data.yaml epochs=15 imgsz=640